In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [4]:
# Get the same results each time
np.random.seed(0)


# Load the training data
data = pd.read_csv("data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

In [5]:
data.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0000,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
1,239607,0.912500,Yet call out all Muslims for the acts of a few...,0.050000,0.237500,0.612500,0.887500,0.1125,0.0,0.0,...,26670,approved,0,0,0,1,0,0.000000,4,80
2,239612,0.830769,This bitch is nuts. Who would read a book by a...,0.107692,0.661538,0.338462,0.830769,0.0000,0.0,0.0,...,26674,rejected,0,0,0,0,0,0.061538,4,65
3,240311,0.968750,You're an idiot.,0.031250,0.062500,0.000000,0.968750,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.000000,0,32
4,240329,0.900000,Who cares!? Stark trek and Star Wars fans are ...,0.100000,0.200000,0.000000,0.900000,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.300000,0,10


In [6]:
comments.info()

<class 'pandas.core.series.Series'>
RangeIndex: 90902 entries, 0 to 90901
Series name: comment_text
Non-Null Count  Dtype 
--------------  ----- 
90902 non-null  object
dtypes: object(1)
memory usage: 710.3+ KB


In [7]:
target.value_counts()

target
1    45451
0    45451
Name: count, dtype: int64

# Задание 1

In [8]:
# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    comments, target,           # данные
    test_size=0.3,  # 30% — тестовая выборка, 70% — обучающая
    random_state=42 # фиксируем случайность для повторяемости
)

# Задание 2

In [9]:
# Создаем экземпляр векторизатора
vectorizer = CountVectorizer()

# Обучаем векторизатор на тренировочных данных и трансформируем их в числовой формат
X_train_vec = vectorizer.fit_transform(X_train)

# Трансформируем тестовые данные (без повторного обучения)
X_test_vec= vectorizer.transform(X_test)

In [10]:
print(f'Размерноть датасета X_train_vec', X_train_vec.shape)
print(f'Размерность датасета y_train', y_train.shape)
print(f'Размерноть датасета X_train_vec', X_test_vec.shape)
print(f'Размерность датасета y_train', y_test.shape)

Размерноть датасета X_train_vec (63631, 57878)
Размерность датасета y_train (63631,)
Размерноть датасета X_train_vec (27271, 57878)
Размерность датасета y_train (27271,)


# Задание 3

In [11]:
# Создаем и обучаем модель логистической регрессии
model = LogisticRegression(max_iter=2000, random_state=42)
model.fit(X_train_vec, y_train)

# Предсказания на тестовой выборке
y_pred = model.predict(X_test_vec)

# Оцениваем точность модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy модели логистической регрессии: {accuracy:.4f}')

Accuracy модели логистической регрессии: 0.9276


# Задание 4

In [12]:
def predict_toxicity(comment):
    # Векторизуем текст
    comment_vec = vectorizer.transform([comment])
    
    # Получаем вероятность принадлежности к классу 1 (токсичный)
    probability = model.predict_proba(comment_vec)[0][1]
    
    if probability > 0.7:
        print(f'Токсичный комментарий: "{comment}"')
        print(f'Вероятность токсичности: {probability:.4f}')
        return probability
    else:
        print(f'Нетоксичный комментарий: "{comment}"')
        print(f'Вероятность токсичности: {probability:.4f}')
        return probability


In [13]:
predict_toxicity("You are loser")

Токсичный комментарий: "You are loser"
Вероятность токсичности: 0.9678


0.9678070844890234

# Задание 5

In [14]:
predict_toxicity("Apples are stupid")

Токсичный комментарий: "Apples are stupid"
Вероятность токсичности: 0.9991


0.9991090380382668

In [15]:
predict_toxicity("I love apples")

Нетоксичный комментарий: "I love apples"
Вероятность токсичности: 0.0587


0.0586974796290112

# Задание 6

In [16]:
# Получаем все слова из словаря
vocab = vectorizer.vocabulary_

# Получаем коэффициенты из модели
coefficients = model.coef_[0]

# Сортируем индексы слов по величине коэффициента
sorted_indices = np.argsort(coefficients)

# Выводим 10 наиболее токсичных слов (с самыми высокими коэффициентами)
print("Топ-10 самых токсичных слов и их коэффициенты:")
for index in sorted_indices[-10:][::-1]:
    word = list(vocab.keys())[list(vocab.values()).index(index)]  # Находим слово по индексу
    print(f"{word}: {coefficients[index]:.4f}")

Топ-10 самых токсичных слов и их коэффициенты:
stupid: 9.2228
idiot: 8.7435
idiots: 8.4722
stupidity: 7.5524
idiotic: 6.8429
crap: 6.5858
dumb: 6.4476
pathetic: 6.4226
hypocrite: 6.3940
moron: 6.3627


# Задание 7

Все слова корректно выбраны,ошибок не обнаружил

# Задание 8

In [17]:
predict_toxicity("I have a christian friend")
predict_toxicity("I have a muslim friend")
predict_toxicity("I have a white friend")
predict_toxicity("I have a black friend")

Нетоксичный комментарий: "I have a christian friend"
Вероятность токсичности: 0.1884
Нетоксичный комментарий: "I have a muslim friend"
Вероятность токсичности: 0.5152
Нетоксичный комментарий: "I have a white friend"
Вероятность токсичности: 0.4054
Нетоксичный комментарий: "I have a black friend"
Вероятность токсичности: 0.5955


0.5954532299859536

Модель этична

# Задание 9

Предвзятость может привести к тому, что даже нейтральное употребление слов с высокой токсичностью усилит агрессивность высказывания. В том числе в нашей модели это способно спровоцировать дискриминацию по религиозному признаку.

# Задание 10

1)Заменить простую фильтрацию по коэффициенту токсичности на систему, учитывающую контекст (например, сарказм, цитирование, обсуждение самого слова). Это снизит количество ложных срабатываний и позволит отличать реальные оскорбления от нейтральных упоминаний.

2)Сдвинуть фокус модели с общего поиска "плохих слов" на выявление системных угроз (например, язык, направленный против меньшинств, инвалидов, ЛГБТК+, женщин.